![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor

# Import any additional modules and start coding below
data = pd.read_csv("rental_info.csv")
print(data.columns)
data['return_date'] = pd.to_datetime(data['return_date'])
data['rental_date'] = pd.to_datetime(data['rental_date'])
data["rental_length_days"] = (data["return_date"] - data["rental_date"]).dt.days

#

data = data.assign(deleted_scenes = [1 if "Deleted Scenes" in a else 0 for a in data['special_features']])
data = data.assign(behind_the_scenes = [1 if "Behind the Scenes" in a else 0 for a in data['special_features']])

# Store in new dataframe called "X"
features =['behind_the_scenes', 'deleted_scenes', 'amount', 'release_year', 'rental_rate', 'length', 'replacement_cost', 'R', 'PG-13', 'amount_2', 'length_2', 'rental_rate_2', 'NC-17', 'PG']
X = data[features]
print(len(X.columns))
y = data["rental_length_days"] 

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=9)

# calling the model with the best parameter
lasso1 = Lasso(alpha=0.00001)
lasso1.fit(X_train, Y_train)

# Using np.abs() to make coefficients positive.  
lasso1_coef = np.abs(lasso1.coef_)

# plotting the Column Names and Importance of Columns. 
print(lasso1_coef)

feature_subset=np.array(features)[lasso1_coef>0.001]
print("Selected Feature Columns: {}".format(feature_subset))


# LinearRegression
reg = LinearRegression().fit(X_train, Y_train)
prediction_lr = reg.predict(X_test)
mse_lr = mean_squared_error(Y_test, prediction_lr) 
print(mse_lr)

#DecisionTreeRegresson()
dtree = DecisionTreeClassifier()
dtree = dtree.fit(X_train, Y_train)
prediction_dtre = dtree.predict(X_test)
mse_dtre = mean_squared_error(Y_test, prediction_dtre) 
print(mse_dtre)

#RandomForestRegressor
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr = regr.fit(X_train, Y_train)
prediction_regr = regr.predict(X_test)
mse_regr = mean_squared_error(Y_test, prediction_regr) 
print(mse_regr)

best_model = reg
best_mse = mse_lr

Index(['rental_date', 'return_date', 'amount', 'release_year', 'rental_rate',
       'length', 'replacement_cost', 'special_features', 'NC-17', 'PG',
       'PG-13', 'R', 'amount_2', 'length_2', 'rental_rate_2'],
      dtype='object')
14
[8.55517188e-02 1.30964790e-01 1.56426789e+00 1.63633140e-02
 1.20117829e+00 3.47624939e-03 3.77422246e-03 4.51626204e-02
 1.44558415e-01 3.79448386e-02 9.48307048e-06 4.89482634e-03
 1.75311619e-01 1.37781883e-01]
Selected Feature Columns: ['behind_the_scenes' 'deleted_scenes' 'amount' 'release_year'
 'rental_rate' 'length' 'replacement_cost' 'R' 'PG-13' 'amount_2'
 'rental_rate_2' 'NC-17' 'PG']
2.9417238646976
3.373148439962181
4.143878747773874
